In [2]:
import pandas as pd
import numpy 
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from openpyxl import Workbook
from openpyxl import load_workbook

data_path = 'E:\\code\\python\\qoe_model\\raw_data\\3w_data.csv'

#############################################################
######## 将 score_name 对应的5种分数选出 choose_num个 #######
#############################################################

def choose_data(data_path, score_name, choose_num):
    
    final_DataFram = pd.DataFrame()
    for i in range(1,6):
        data = pd.read_csv(data_path)
        data = data.drop_duplicates(subset=None, keep='first' , inplace=False)
        data = data.loc[data[score_name] == i]
        data = data.reset_index(drop = True)
        row = data.iloc[:,0].size - 1
        #data = data.loc[np.random.randint(0, row, choose_num).tolist()]
        data = data.loc[range(0,choose_num)]
        
        final_DataFram = pd.concat([final_DataFram, data])
    final_DataFram = final_DataFram.reset_index(drop = True)
        
    return final_DataFram

##############################################################
################### 从 excel 中读入数据 ######################
##############################################################
def import_data(filename):

    wb = load_workbook(filename)
    ws = wb.get_sheet_by_name("Sheet1")
    row = ws.max_row
    col = ws.max_column
    arr1 = numpy.zeros( (row-1,col) )
    
    for row_A in range(2, row):
        for col_A in range(1, col + 1):
            arr1[row_A-2,col_A-1] = ws.cell(row = row_A, column = col_A).value
    return arr1


def dataFrame_to_xlsx(df, path):
    writer = pd.ExcelWriter(path, engine='xlsxwriter')
    df.to_excel(writer, sheet_name='Sheet1')
    workbook  = writer.book
    worksheet = writer.sheets['Sheet1']
    writer.save()

####################################################################
######################## 将 data 复制 i 次 #########################
####################################################################

def duplicate(data, i):
    final_DataFrame = pd.DataFrame()
    for j in range(i):
        final_DataFrame = pd.concat([final_DataFrame, data])
    return final_DataFrame

#USER_SCORE = choose_data(data_path, 'USER_SCORE',500)
#USER_SCORE = USER_SCORE.drop_duplicates(subset=None, keep='first' , inplace=False) 
#print(USER_SCORE['USER_SCORE'].value_counts())
#USER_SCORE.to_excel('E:\\code\\python\\qoe_model\\helloExcel.xls')
#df['USER_SCORE'].value_counts()


#####################################################################
#################### 将 name 列的离散数据进行编号 ###################
#####################################################################

def class_normalization(name, X):
    
    a = X[name]
    b = a.value_counts()
    c = b.index

    list1 = []
    list2 = []
    for i in range(len(c)):
        list1.append(i)
        list2.append(c[i])
    return a.replace(list2, list1)

#name = 'VIDEO_CLARITY'
#a = class_normalization(name, X_train)

#####################################################################
########################### 移除 name 列 ############################
#####################################################################
def remove_col(name, all_name):
    
    list = []
    for i in range(len(column_names)):
        if column_names[i] != name:
            list.append(column_names[i])
    return list

In [ ]:
#################################################################################################
################# 将 USER_SCORE 不同分值的数据复制不同的次数，使得所有分值近均匀分布 ############
#################################################################################################

data = pd.read_csv(data_path)
data = data.drop_duplicates(subset = None, keep = 'first' , inplace = False)
data1 = data.loc[data['USER_SCORE'] == 1]
data2 = data.loc[data['USER_SCORE'] == 2]
data3 = data.loc[data['USER_SCORE'] == 3]
data4 = data.loc[data['USER_SCORE'] == 4]
data5 = data.loc[data['USER_SCORE'] == 5]

score1 = duplicate(data1,30)
score2 = duplicate(data2, 13)
score3 = duplicate(data3, 4)
score4 = duplicate(data4, 3)

expand_data = pd.DataFrame()
expand_data = pd.concat([expand_data, score1, score2, score3, score4, data5])

from sklearn.utils import shuffle
df = shuffle(expand_data)

##################################################################################################
########## 对 SCREEN_RESOLUTION_LONG 做编号，同时对 VIDEO_CLARITY 进行归一化 #####################
##################################################################################################

col_X = ['InitialBufferTime', 'VideoPlayDuration','StallingRatio', 'VIDEO_BITRATE', 'VIDEO_CLARITY', 
       'VIDEO_ALL_PEAK_RATE', 'VIDEO_AVERAGE_RATE', 'USERBUFFERTIME', 'VIDEOSIZE', 'SCREEN_RESOLUTION_LONG',
       'VIDEO_BUFFERING_PEAK_RATE']

col = ['InitialBufferTime', 'VideoPlayDuration','StallingRatio', 'VIDEO_BITRATE', 'VIDEO_CLARITY', 
       'VIDEO_ALL_PEAK_RATE', 'VIDEO_AVERAGE_RATE', 'USERBUFFERTIME', 'VIDEOSIZE',
       'VIDEO_BUFFERING_PEAK_RATE']

col_y = ['EVMOS', 'ELOADING', 'ESTALLING', 'USER_SCORE']

data = pd.read_csv('E:\\code\\python\\qoe_model\\raw_data\\3w_data.csv')

#data1 = data[col]
data1 = data['SCREEN_RESOLUTION_LONG']

data2 = data[col_X]
data2 = data2.drop(['SCREEN_RESOLUTION_LONG'],axis=1)

score = data[col_y]
#data1['VIDEO_CLARITY'].value_counts()

data['VIDEO_CLARITY'].loc[data['VIDEO_CLARITY'] == 240] = 2
data['VIDEO_CLARITY'].loc[data['VIDEO_CLARITY'] == 360] = 3
data['VIDEO_CLARITY'].loc[data['VIDEO_CLARITY'] == 720] = 4
data['VIDEO_CLARITY'].loc[data['VIDEO_CLARITY'] == 800] = 5
data['VIDEO_CLARITY'].loc[data['VIDEO_CLARITY'] == 1080] = 6

data = data.drop(col, axis=1)

from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
data2 = ss.fit_transform(data2)

data2 = data.drop(['SCREEN_RESOLUTION_LONG'],axis=1)

data2.insert(1, 'SCREEN_RESOLUTION_LONG', data1)

dataFrame_to_xlsx(data2, 'E:\\code\\python\\qoe_model\\processed_data.xlsx')